In [12]:
from pubnub.pnconfiguration import PNConfiguration
from pubnub.pubnub import PubNub, SubscribeListener
from pubnub.enums import PNStatusCategory

pnconfig = PNConfiguration()
pnconfig.subscribe_key="sub-c-b1cadece-f0fa-11e3-928e-02ee2ddab7fe"
# for twitter: subkey: "sub-c-78806dd4-42a6-11e4-aed8-02ee2ddab7fe", channel: "pubnub-twitter"
# for game: subkey: "sub-c-b1cadece-f0fa-11e3-928e-02ee2ddab7fe", channel: "pubnub-game-state"
pnconfig.ssl = False

pubnub = PubNub(pnconfig)

In [14]:
my_listener = SubscribeListener()
pubnub.add_listener(my_listener)

a = pubnub.subscribe().channels('pubnub-game-state').execute()
my_listener.wait_for_connect()
print('connected')

def sampling(a, b, total):
    data = []
    hashMap = [[] for bucket in range(b)]
    for i in range(0, total):
        result = my_listener.wait_for_message_on('pubnub-game-state')
        hashkey = hash(result.message["player_id"]) % len(hashMap)
        filter_condition = filter_data(result, 'action_type', ['Instant', 'AoE'])
        if hashkey <= a and filter_condition:
            data.append(result.message)
    return data

def filter_data(data, var, condition):
    b = [0] * 10
    hash_data = hash(data.message[var]) % 10
    for i in condition:
        hash_condition = hash(i) % 10
        b[hash_condition] = 1
    if (b[hash_data] == 1):
        return True


connected
